# LRS3 Extract Images from video

## Imports

In [1]:
from pathlib import Path
from tqdm import tqdm
from more_itertools import ilen
from collections import defaultdict
import math
from multiprocessing import Pool

In [2]:
data_dir = Path("/mnt/U/Datasets/lrs3pretrain/pretrain/")
images_dir = Path("/mnt/U/Datasets/lrs3pretrain/pretrain_images/")

In [3]:
speakers = list(data_dir.glob("*"))
utt_per_spk = {x.name: list(x.glob("*.mp4")) for x in speakers}
n_utt_per_spk = {x: len(y) for x, y in utt_per_spk.items()}
n_utterances = sum(n_utt_per_spk.values())

In [4]:
def get_out_path(spk_utt):
    spk, utt = spk_utt
    out_path = images_dir / spk / (utt.stem + "_%d.jpg")
    if not out_path.parent.exists():
        out_path.parent.mkdir(parents=True, exist_ok=True)
    return out_path

def extract_images(utt_outpath):
    utt, out_path = utt_outpath
    !ffmpeg -i {utt} -s 224x224 -vf fps=0.25 {out_path} -loglevel quiet

In [5]:
spk_utts = []
for spk, utts in utt_per_spk.items():
    spk_utts += [(spk, utt) for utt in utts]
paths = list(map(get_out_path, spk_utts))
spk_paths = list(zip([x[1] for x in spk_utts], paths))

In [ ]:
p = Pool(16)
list(tqdm(p.imap_unordered(extract_images, spk_paths), total=n_utterances))